In [14]:
import pandas as pd
import numpy as np
import os
import nltk
import seaborn as sns
from matplotlib import pyplot as plt
import transformers
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.model_selection import train_test_split

from itertools import chain
from transformers import OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer

In [323]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.chdir("/Users/mm06832/Documents/Classes/CS585/final_project/chatbot-with-personality/data")
df = pd.read_csv("processed_data_final.csv", low_memory = False)
df =  df[df.columns[1:]]

In [325]:
df

,sentence_1,sentence_1_id,sentence_2,sentence_2_id,speaker_1,speaker_1_id,speaker_2,speaker_2_id,movie_id,history,...,thriller,short,western,documentary,horror,animation,film-noir,music,war,counts
0,Can we make this quick? Roxanne Korrine and A...,L194,"Well, I thought we'd start with pronunciation,...",L195,BIANCA,u0,CAMERON,u2,m0,[],...,0,0,0,0,0,0,0,0,0,3
1,"Well, I thought we'd start with pronunciation,...",L195,Not the hacking and gagging and spitting part....,L196,BIANCA,u0,CAMERON,u2,m0,['Can we make this quick? Roxanne Korrine and...,...,0,0,0,0,0,0,0,0,0,3
2,Not the hacking and gagging and spitting part....,L196,Okay... then how 'bout we try out some French ...,L197,BIANCA,u0,CAMERON,u2,m0,['Can we make this quick? Roxanne Korrine and...,...,0,0,0,0,0,0,0,0,0,3
3,You're asking me out. That's so cute. What's ...,L198,Forget it.,L199,BIANCA,u0,CAMERON,u2,m0,[],...,0,0,0,0,0,0,0,0,0,1
4,"No, no, it's my fault -- we didn't have a prop...",L200,Cameron.,L201,BIANCA,u0,CAMERON,u2,m0,[],...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221611,"Your orders, Mr Vereker?",L666369,I'm to take the Sikali with the main column to...,L666370,DURNFORD,u9030,VEREKER,u9034,m616,[],...,0,0,0,0,0,0,0,0,1,3
221612,I'm to take the Sikali with the main column to...,L666370,Lord Chelmsford seems to want me to stay back ...,L666371,DURNFORD,u9030,VEREKER,u9034,m616,"['Your orders, Mr Vereker?']",...,0,0,0,0,0,0,0,0,1,3
221613,Lord Chelmsford seems to want me to stay back ...,L666371,I think Chelmsford wants a good man on the bor...,L666372,DURNFORD,u9030,VEREKER,u9034,m616,"['Your orders, Mr Vereker?', ""I'm to take the ...",...,0,0,0,0,0,0,0,0,1,3
221614,"Well I assure you, Sir, I have no desire to cr...",L666520,"And I assure you, you do not In fact I'd be ob...",L666521,DURNFORD,u9030,VEREKER,u9034,m616,[],...,0,0,0,0,0,0,0,0,1,2


In [ ]:
#### don't run yet, still WIP. start from below instead

def build_inputs(persona, history, reply):
    # Build our sequence by adding delimiters and concatenating

    sequence = [[bos] + list(chain(*persona))] + history + [reply + [eos]]
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                    for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                    for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens
    return words, segments, position, sequence

def pad(x, padding):
    return x + [padding] * (padding_length - len(x))

(words_1, words_2,
 segments_1, segments_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>'))
                                   for x in (words_1, words_2,
                                             segments_1, segments_2)]

def processTrainingData(df, index):
    
    bos, eos, speaker1, speaker2 = "<bos>", "<eos>", "<speaker1>", "<speaker2>"
    
    # Let's define our contexts and special tokens
    persona = [["i", "like", "playing", "football", "."],
               ["i", "am", "from", "NYC", "."]]
    history = [["hello", "how", "are", "you", "?"],
               ["i", "am", "fine", "thanks", "."]]
    correct_reply = X_train.iloc[0]['sentence_1'].split()
    incorrect_reply = X_train.iloc[2]['sentence_1'].split()
    
    words_1, segments_1, position_1, sequence_1 = build_inputs(persona, history, correct_reply)
    words_2, segments_2, position_2, sequence_2 = build_inputs(persona, history, incorrect_reply)

    # Tokenize words and segments embeddings:
    words_1 = tokenizer.convert_tokens_to_ids(words_1) 
    words_2 = tokenizer.convert_tokens_to_ids(words_2)
    segments_1 = tokenizer.convert_tokens_to_ids(segments_1)
    segments_2 = tokenizer.convert_tokens_to_ids(segments_2)
    
    lm_targets_1 = ([-1] * sum(len(s) for s in sequence[:-1])) \
             + [-1] + tokenizer.convert_tokens_to_ids(sequence[-1][1:])
    lm_targets_2 = [-1] * len(words_2)
    
    # Store the position of the last tokens for the next-sentence prediction loss
    last_token_1 = len(words_1) - 1
    last_token_2 = len(words_2) - 1

    # Now we can pad reply and distractor inputs and targets to the same length
    padding_length = max(len(words), len(words_2))
    def pad(x, padding):
        return x + [padding] * (padding_length - len(x))

    (words_1, words_2,
     segments_1, segments_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>'))
                                       for x in (words_1, words_2,
                                                 segments_1, segments_2)]

    (lm_targets_1, lm_targets_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>')) for x in (lm_targets_1, lm_targets_2)]
    
    batch_seq_len = 1

    # And gather reply and distractor inputs to build the input tensors:
    # words tokens
    input_ids = torch.tensor([words_1, words_2]).unsqueeze(batch_seq_len - 1)
    # segment tokens
    token_type_ids = torch.tensor([segments_1, segments_2]).unsqueeze(batch_seq_len - 1)
    # Last tokens location
    mc_token_ids = torch.tensor([last_token_1, last_token_2], dtype=torch.long).unsqueeze(batch_seq_len - 1)
    # Language modeling labels
    lm_labels = torch.tensor([lm_targets_1, lm_targets_2], dtype=torch.long).unsqueeze(batch_seq_len - 1)
    # Next-sentence prediction labels
    mc_labels = torch.zeros(batch_seq_len, dtype=torch.long) # Gold reply is 1st (index 0)

In [17]:
#stratify train test for conversations with more than one exchange to prevent target leakage
df = df.assign(counts=df.groupby('index')['index'].transform('count'))
df_1 = df[df['counts'] == 1]
df_2 = df[df['counts'] != 1]

In [18]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df_1.drop('sentence_2', axis=1), df_1['sentence_2'])
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_2.drop('sentence_2', axis=1), df_2['sentence_2'],stratify=df_2['index'])

In [19]:
X_train = pd.concat([X_train_1, X_train_2])
X_test = pd.concat([X_test_1, X_test_2])
y_train = pd.concat([y_train_1, X_train_2])
y_test = pd.concat([y_test_1, y_test_2])

In [360]:
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2DoubleHeadsModel.from_pretrained('gpt2')

In [348]:
SPECIAL_TOKENS = {"bos_token": "<bos>", "eos_token": "<eos>", "additional_special_tokens": ["<speaker1>", "<speaker2>"], "pad_token": "<pad>"}

tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer))

Embedding(50262, 768)

In [349]:
# Let's define our contexts and special tokens
persona = [["i", "like", "playing", "football", "."],
           ["i", "am", "from", "NYC", "."]]
history = [["hello", "how", "are", "you", "?"],
           ["i", "am", "fine", "thanks", "."]]
correct_reply = X_train.iloc[0]['sentence_1'].split()#["great", "to", "hear"]
incorrect_reply = X_train.iloc[2]['sentence_1'].split() #["i", "hate", "you"]
bos, eos, speaker1, speaker2 = "<bos>", "<eos>", "<speaker1>", "<speaker2>"

def build_inputs(persona, history, reply):
    # Build our sequence by adding delimiters and concatenating
    
    sequence = [[bos] + list(chain(*persona))] + history + [reply + [eos]]
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens
    return words, segments, position, sequence

In [350]:
words_1, segments_1, position_1, sequence_1 = build_inputs(persona, history, correct_reply)
words_2, segments_2, position_2, sequence_2 = build_inputs(persona, history, incorrect_reply)

In [351]:
words_1

['<bos>',
 'i',
 'like',
 'playing',
 'football',
 '.',
 'i',
 'am',
 'from',
 'NYC',
 '.',
 '<speaker1>',
 'hello',
 'how',
 'are',
 'you',
 '?',
 '<speaker2>',
 'i',
 'am',
 'fine',
 'thanks',
 '.',
 '<speaker1>',
 'Almost',
 "didn't",
 'recognize',
 'you',
 'with',
 'your',
 'hair',
 'like',
 'that.',
 'How',
 'you',
 'been?',
 '<eos>']

In [352]:
# Tokenize words and segments embeddings:
words_1 = tokenizer.convert_tokens_to_ids(words_1) 
words_2 = tokenizer.convert_tokens_to_ids(words_2)
segments_1 = tokenizer.convert_tokens_to_ids(segments_1)
segments_2 = tokenizer.convert_tokens_to_ids(segments_2)

In [359]:
segments_1


[50259,
 50259,
 50259,
 50259,
 50259,
 50259,
 50259,
 50259,
 50259,
 50259,
 50259,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50259,
 50259,
 50259,
 50259,
 50259,
 50259,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50260,
 50261,
 50261,
 50261]

In [353]:
lm_targets_1 = ([-1] * sum(len(s) for s in sequence[:-1])) \
             + [-1] + tokenizer.convert_tokens_to_ids(sequence[-1][1:])
lm_targets_2 = [-1] * len(words_2)

In [357]:
# Store the position of the last tokens for the next-sentence prediction loss
last_token_1 = len(words_1) - 1
last_token_2 = len(words_2) - 1

# Now we can pad reply and distractor inputs and targets to the same length
padding_length = max(len(words), len(words_2))
def pad(x, padding):
    return x + [padding] * (padding_length - len(x))

(words_1, words_2,
 segments_1, segments_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>'))
                                   for x in (words_1, words_2,
                                             segments_1, segments_2)]

(lm_targets_1, lm_targets_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>')) for x in (lm_targets_1, lm_targets_2)]

In [355]:
batch_seq_len = 1

# And gather reply and distractor inputs to build the input tensors:
# words tokens
input_ids = torch.tensor([words_1, words_2]).unsqueeze(batch_seq_len - 1)
# segment tokens
token_type_ids = torch.tensor([segments_1, segments_2]).unsqueeze(batch_seq_len - 1)
# Last tokens location
mc_token_ids = torch.tensor([last_token_1, last_token_2], dtype=torch.long).unsqueeze(batch_seq_len - 1)
# Language modeling labels
lm_labels = torch.tensor([lm_targets_1, lm_targets_2], dtype=torch.long).unsqueeze(batch_seq_len - 1)
# Next-sentence prediction labels
mc_labels = torch.zeros(batch_seq_len, dtype=torch.long) # Gold reply is 1st (index 0)

In [356]:
output = model(input_ids, mc_token_ids=mc_token_ids, mc_labels = mc_labels, token_type_ids = token_type_ids, lm_labels = lm_labels)
lm_loss = output[0]
mc_loss = output[1] 
lm_scores = output[2]
mc_scores = output[3]

lm_coef = 2.0
mc_coef = 1.0

total_loss = lm_loss * lm_coef + mc_loss * mc_coef
total_loss

tensor(40.5495, grad_fn=<AddBackward0>)

#### Custom Dataloader

In [ ]:
#WIP - to load data for training 

from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import random

class ConvDataset(Dataset, ):
    """Movie dialogue conversation dataset."""
    
    def build_inputs(persona, history, reply):
        # Build our sequence by adding delimiters and concatenating

        sequence = [[bos] + list(chain(*persona))] + history + [reply + [eos]]
        sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                        for i, s in enumerate(sequence[1:])]
        # Build our word, segments and position inputs from the sequence
        words = list(chain(*sequence))                          # word tokens
        segments = [speaker2 if i % 2 else speaker1             # segment tokens
                        for i, s in enumerate(sequence) for _ in s]
        position = list(range(len(words)))                      # position tokens
        return words, segments, position, sequence

    def pad(x, padding):
        return x + [padding] * (padding_length - len(x))

    (words_1, words_2,
     segments_1, segments_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>'))
                                       for x in (words_1, words_2,
                                                 segments_1, segments_2)]
    
    def __getTest__():
        return (self.y_train, self.y_test)
    
    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        df = pd.read_csv(csv_file, low_memory = False)
        df =  df[df.columns[1:]]
        df = df.assign(counts=df.groupby('index')['index'].transform('count'))
        
        df_1 = df[df['counts'] == 1]
        df_2 = df[df['counts'] != 1]
        
        X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df_1.drop('sentence_2', axis=1), df_1['sentence_2'])
        X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_2.drop('sentence_2', axis=1), df_2['sentence_2'],stratify=df_2['index'])
        
        self.X_train = pd.concat([X_train_1, X_train_2])
        self.X_test = pd.concat([X_test_1, X_test_2])
        self.y_train = pd.concat([y_train_1, X_train_2])
        self.y_test = pd.concat([y_test_1, y_test_2])
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        persona = [["i", "like", "playing", "football", "."],
                   ["i", "am", "from", "NYC", "."]]
        history = [["hello", "how", "are", "you", "?"],
                   ["i", "am", "fine", "thanks", "."]]
        correct_reply = self.X_train.iloc[0]['sentence_1'].split()
        incorrect_index = random.sample(set(range(len(self.X_train))) - set(idx))
        incorrect_reply = self.X_train.iloc[incorrect_index]['sentence_1'].split()

        words_1, segments_1, position_1, sequence_1 = build_inputs(persona, history, correct_reply)
        words_2, segments_2, position_2, sequence_2 = build_inputs(persona, history, incorrect_reply)

        # Tokenize words and segments embeddings:
        words_1 = tokenizer.convert_tokens_to_ids(words_1) 
        words_2 = tokenizer.convert_tokens_to_ids(words_2)
        segments_1 = tokenizer.convert_tokens_to_ids(segments_1)
        segments_2 = tokenizer.convert_tokens_to_ids(segments_2)

        lm_targets_1 = ([-1] * sum(len(s) for s in sequence[:-1])) \
                 + [-1] + tokenizer.convert_tokens_to_ids(sequence[-1][1:])
        lm_targets_2 = [-1] * len(words_2)

        # Store the position of the last tokens for the next-sentence prediction loss
        last_token_1 = len(words_1) - 1
        last_token_2 = len(words_2) - 1

        # Now we can pad reply and distractor inputs and targets to the same length
        padding_length = max(len(words), len(words_2))
        def pad(x, padding):
            return x + [padding] * (padding_length - len(x))

        (words_1, words_2,
         segments_1, segments_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>'))
                                           for x in (words_1, words_2,
                                                     segments_1, segments_2)]

        (lm_targets_1, lm_targets_2) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>')) for x in (lm_targets_1, lm_targets_2)]

        batch_seq_len = 1

        # And gather reply and distractor inputs to build the input tensors:
        # words tokens
        input_ids = torch.tensor([words_1, words_2]).unsqueeze(batch_seq_len - 1)
        # segment tokens
        token_type_ids = torch.tensor([segments_1, segments_2]).unsqueeze(batch_seq_len - 1)
        # Last tokens location
        mc_token_ids = torch.tensor([last_token_1, last_token_2], dtype=torch.long).unsqueeze(batch_seq_len - 1)
        # Language modeling labels
        lm_labels = torch.tensor([lm_targets_1, lm_targets_2], dtype=torch.long).unsqueeze(batch_seq_len - 1)
        # Next-sentence prediction labels
        mc_labels = torch.zeros(batch_seq_len, dtype=torch.long) # Gold reply 

        return sample

#### Custom Training Loop

In [ ]:
#retrain only final fc layer for language modeling task

for param in model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.lm_head.in_features
model.lm_head = nn.Linear(num_ftrs, vocab_size)

model = model.to(device)

model.eval()
#criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
#optimizer_conv = optim.SGD(model.lm_head.parameters(), lr=0.001, momentum=0.9)

#optimizer_conv

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)